# Data munging

Most of the time and effort in projects using computational methods goes into data munging, the process of gathering, cleaning, and preparing data for our actual analysis. This notebook walks through that process with the OKCupid public profile data. You may either:
1. Step through the code one cell at a time, readign the descriptions to learn how it works, or
2. Run all the code here to prepare data for the other labs / notebooks. 
    - **Note:** You should check for errors after you run this. Common errors and what to do about them are documented below. 
    
@Author: [Jeff Lockhart](http://www-personal.umich.edu/~jwlock/)

## 0. Import libraries we'll use

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os.path

## 1. Download the data
- This code checks whether you have downloaded the data yet.
- If you have not, it downloads the data. 

In [ ]:
if os.path.isfile("data/profiles.csv.zip"):
    print("You already downloaded the data. Great!")
else:
    print('Downloading file...')
    urllib.request.urlretrieve(url="https://github.com/rudeboybert/JSE_OkCupid/raw/master/profiles.csv.zip",
                               filename="data/profiles.csv.zip")

## 2. Read the data into python using pandas

In [ ]:
profiles = pd.read_csv('data/profiles.csv.zip')

## 3. Selecting 18-59 year old users
- Here we select within the profiles using the square brakcets `[]`.
- We select cases where two things are true: `age < 60` and `age > 17`. The round brackets `()` are used to group our conditions, so that python knows `&` doesn't just mean the two things closest to it, i.e. `60 & profiles.age` (that code would not make sense).
- We save the result, our list of profiles that meet those conditions, as our new `profiles` variable with the `=`.

In [ ]:
print("We started with", profiles.shape[0], "profiles.")

profiles = profiles[(profiles.age < 60) & (profiles.age > 17)]

print("After selecting just those between 18 and 59, we have", 
      profiles.shape[0], "left.")

## 4. Grouping people by age

### Defining functions to recode data
- To change someone's exact age to their age group, we want to do something to the number. We can define the thing we want to do as a `function`. 
- Here are two functions that take some age `a` and convert it to an age group using these steps:
    1. Divide someone's age by 10 (e.g. 34 becomes 3.4)
    2. Convert that age to an integer (e.g. 3.4 becomes 3)
    3. Multiply that integer by 10 (e.g. 3 becomes 30)
    4. Turn that integer into a string / text 
- `group_age()` and `group_age_long()` do exactly the same thing, but the long one writes out each step so it is easier to follow.
- The last line gives the function the number 34. As we wanted, it prints the result `'30'`.

In [ ]:
def group_age_long(a):
    g = a / 10    #divide age by 10
    g = int(g)    #convert age to an integer 
    g = g * 10    #multiply age by 10
    g = str(g)    #convert the number to text / string
    return g      #return the result

def group_age(a):
    return str(int(a/10)*10)

group_age(34)

### Apply
- Apply is a powerful tool built into pandas. The logic is simple. Each person has an age, and we'd like to `apply` our age-grouping function to each one of those ages. 
- This code says "go to the profiles data, look at the age column in it, and apply `group_age` to each of the things there.
- We create a new column in the profiles data called `age_group`, and we set it equal to the result of the `group_age` function.
- We then print the head of the relevant columns in order to check that we did it correctly.

In [ ]:
profiles['age_group'] = profiles.age.apply(group_age)
profiles[['age', 'age_group']].head(10)

## 5. Grouping people by height
1. We define a function, `height()`, that returns `under_6` if someone is under 6 feet (72 inches) tall and `over_6` otherwise.
2. We apply that function to the column of our data that contains users' height.
3. We save the result in a new column called `height_group`.

In [ ]:
def height(inches):
    h = 'under_6'
    if inches >= 72:
        h = 'over_6'
    return h

profiles['height_group'] = profiles.height.apply(height)
profiles[['height', 'height_group']].head(10)

## 6. Merging text
- The OKC data has 10 different columns with profile text, one for each long-answer question in users' profiles. We want to look at all of the profile text, so this merges it all together in a new column called `text`.

In [ ]:
# which columns have text in them
essay_cols = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 
              'essay7', 'essay8', 'essay9']

def concat(row, cols):
    tmp = []
    for c in cols:
        tmp.append(str(row[c]))
    new = '\n'.join(tmp)
    return new

profiles['text'] = profiles.apply(concat, axis=1, cols=essay_cols)

## 7. Grouping categories
- Much of our data comes in many or complicated categories. For example, there are more than 20 different levels of education in the education column. 
- We want to group these together so that they are easier to interpret

#### This cell defines what categories we want and which people belong in them.
- Each column we will recode has been given its own dictionary.
- The keys of that dictionary are the new categories we want to make.
- The value of each key is a list of values that should all be combined in that category.
- In many cases, I have left out a default category to simplify the code.

In [ ]:
# education
ed_levels = {'<HS': ['dropped out of high school', 'working on high school'],
             'HS': ['graduated from high school', 'working on college/university', 
                    'two-year college', 'dropped out of college/university', 
                    'high school'], 
             'BA': ['graduated from college/university', 
                    'working on masters program', 'working on ph.d program', 
                    'college/university', 'working on law school', 
                    'dropped out of masters program', 
                    'dropped out of ph.d program', 'dropped out of law school', 
                    'dropped out of med school'],
             'Grad_Pro': ['graduated from masters program',
                          'graduated from ph.d program',                           
                          'graduated from law school', 
                          'graduated from med school', 'masters program', 
                          'ph.d program', 'law school', 'med school']
            }

#body type
bodies = {'average': ['average'], 
          'fit': ['fit', 'athletic', 'jacked'], 
          'thin': ['thin', 'skinny'], 
          'overweight': ['curvey', 'a little extra', 'full figured', 'overweight']
         }

# smoking
smoke = {'no': ['no'], np.nan: ['nan']}

# Has children
kids = {'yes': ['has a kid', 'has kids']}

#has pets
has_pets = {'yes': ['has']}

# alcohol use
drinks = {'no': ['rarely', 'not at all']}

# drug use
drugs = {'no': ['never']}

# Employment sector
jobs = {'education': ['student', 'education'], 
        'STEM': ['science', 'computer'], 
        'business': ['sales', 'executive', 'banking'], 
        'creative': ['artistic', 'entertainment'], 
        'med_law': ['medicine', 'law'],
        np.nan: ['nan']
       }

# religion 
religion = {'none': ['agnosticism', 'atheism'],
            'catholicism': ['catholicism'],
            'christianity': ['christianity'],
            'judaism': ['judaism'],
            'buddhism': ['buddhism'],
            np.nan: ['nan']
           }

# languages spoken. People who list more than one langage have a comma between them
# thus anyone with a comma in the languages column has more than one language
languages = {'multiple': [',']}

# race/ethnicity for exact matching
ethn = {'White': ['white', 'middle eastern', 'middle eastern, white'], 
        'Asian': ['asian', 'indian', 'asian, pacific islander'], 
        'Black': ['black']
       }   

# race/ethnicityfor fuzzy matching
ethn2 = {'Latinx': ['latin'], 'multiple': [','], np.nan: ['nan']}   

#### Functions for figuring out which group someone belongs in

In [ ]:
def recode(text, dictionary, default=np.nan):
    '''Function for recoding categories in a column based on exact matches'''
    out = default
    text = str(text)
    
    for x in dictionary.keys():
        for y in dictionary[x]:
            if y == text: #exact match
                out = x
                return out
    return out

def recode_fuzzy(text, dictionary, default=np.nan):
    '''Function for recoding categories in a column based on partial matches'''
    out = default
    text = str(text)
    
    for x in dictionary.keys():
        for y in dictionary[x]:
            if y in text: #partial match
                out = x
                return out
    return out

#### Use the functions and the categories we defined to recode the variables

In [ ]:
profiles['edu'] = profiles.education.apply(recode, dictionary=ed_levels, 
                                            default='unknown')
profiles['kids'] = profiles.offspring.apply(recode_fuzzy, dictionary=kids, 
                                            default='no')
profiles['smoker'] = profiles.smokes.apply(recode, dictionary=smoke, 
                                            default='yes')
profiles['body'] = profiles.body_type.apply(recode, dictionary=bodies, 
                                            default='unknown')
profiles['alcohol_use'] = profiles.drinks.apply(recode, dictionary=drinks, 
                                            default='yes')
profiles['drug_use'] = profiles.drugs.apply(recode, dictionary=drugs, 
                                            default='yes')
profiles['industry'] = profiles.job.apply(recode_fuzzy, dictionary=jobs, 
                                            default='other')
profiles['religion'] = profiles.religion.apply(recode_fuzzy, dictionary=religion, 
                                            default='other')
profiles['languages'] = profiles.speaks.apply(recode_fuzzy, dictionary=languages, 
                                            default='English_only')

In [ ]:
profiles[['education', 'edu']].head(10)

#### More complex recoding: race and ethnicity
- Recoding race/ethnicity is more complex than most other variables. Indeed, it is a hotly debated topic in social science. 
- Here, we use a combination of exact matching and fuzzy matching to approximate the 2010 US Census categories.
    - This function deviates from the census by creating exclusive Latinx category. 
    - Selecting just 'latin' and nothing else was the 3rd most frequent ethnicity in this data. 
    - The discision to include people who identified as both 'latin' and another race in 'Latinix' is based in research on Latinx people's experience with the US Census, but like all racial and ethnic categorization systems, it is flawed.

In [ ]:
def census_2010_ethnicity(t):
    text = str(t)
    e = recode(text, ethn, default='other')
    if 'other' == e:
        e = recode_fuzzy(text, ethn2, default='other')
    return e

profiles['race_ethnicity'] = profiles.ethnicity.apply(census_2010_ethnicity)

#### Another odd case: pets
- The pets data is a bit different than all the other data, so it was easier to write a separate function for it.

In [ ]:
def which_pets(t, criterion='has'):
    '''Function for determining which pets someone has or likes'''
    d = False
    c = False
    t = str(t)
    p = 'neither'
    if t == 'nan':
        p = np.nan
    
    if 'has dogs' in t:
        d = True
    if 'has cats' in t:
        c = True
        
    if criterion == 'likes':
        if 'likes dogs' in t:
            if 'dislikes dogs' not in t:
                d = True
        if 'likes cats' in t:
            if 'dislikes cats' not in t:
                c = True
        
    if c and d:
        p = 'both'
    elif c:
        p = 'cats'
    elif d:
        p = 'dogs'
        
    return p

In [ ]:
profiles['pets_likes'] = profiles.pets.apply(which_pets, criterion='likes')
profiles['pets_has'] = profiles.pets.apply(which_pets, criterion='has')
profiles['pets_any'] = profiles.pets.apply(recode_fuzzy, dictionary=has_pets, 
                                            default='no')

## 8. Dropping unneeded columns
- We created a lot of new columns so far, and we're not interested in all of the old columns. 
- This code selects just the columns we want to keep.

In [ ]:
print('Initial columns:\n', profiles.columns.values)
profiles = profiles[['age_group', 'age', 'body', 'alcohol_use', 'drug_use', 'edu', 
                     'race_ethnicity', 'height_group', 'industry', 'kids', 
                     'orientation', 'pets_likes', 'pets_has', 'pets_any', 
                     'religion', 'sex', 'smoker', 'languages', 'text', 'essay0', 
                     'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 
                     'essay7', 'essay8', 'essay9']]
print('Final columns:\n', profiles.columns.values)

## 9. Save the results
This cell saves the cleaned up data to a file so we can use it again later.

In [ ]:
profiles.to_csv('data/clean_profiles.tsv', sep='\t', index=False)